### Configuração inicial

In [66]:
import pathlib
import sys
if str(pathlib.Path().resolve().name) == "AP1":
    root = str(pathlib.Path().resolve().parent)+'/'
    sys.path.append(root)
    requirements_path = root + 'requirements.txt'
else:
    ! git clone https://github.com/Fabio-Trindade/Eng-Aprendizado-Maquina.git
    root = str(pathlib.Path().resolve())
    src_path = root + '/Eng-Aprendizado-Maquina/'
    requirements_path = src_path + 'requirements.txt'
    sys.path.append(root + '/Eng-Aprendizado-Maquina/')

! pip install -r $requirements_path

### Importando Bibliotecas Necessárias

In [73]:
from src.utils.util_path import UtilPath
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import joblib


ModuleNotFoundError: No module named 'sklearn.external'

In [44]:
root_project = UtilPath.get_root_path()
print(root_project)

/home/fabio/Mestrado/Eng-Aprendizado-Maquina


### Obtendo Dados

In [45]:
df = pd.read_csv(root_project + '/datasets/student_records/student_records.csv')

In [46]:
display(df)

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


## Preparação dos Dados

### Obtendo features de treino

In [47]:
feature_columns = ['OverallGrade', 'Obedient', 'ResearchScore','ProjectScore']
features = df[feature_columns]


In [48]:
features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


### Obtendo labels/targets


In [49]:
label_column = ['Recommend']
labels = df[label_column]

In [50]:
labels

,Recommend
0,Yes
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No


### Separando em features categóricas

In [51]:
categorical_columns =  ['OverallGrade', 'Obedient']

### Separando em features numéricas


In [52]:
numeric_columns =  ['ResearchScore', 'ProjectScore']

### Normalizando features numéricas

In [53]:
ss = StandardScaler()

ss.fit(features[numeric_columns])

features[numeric_columns] = ss.transform(features[numeric_columns])

/tmp/ipykernel_331729/3433130785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[numeric_columns] = ss.transform(features[numeric_columns])


In [54]:
features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


### Transformando dados categóricos

In [55]:
features = pd.get_dummies(features,
columns=categorical_columns)


In [56]:
features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,True,False,False,False,False,False,True
1,0.730648,-0.091777,False,False,True,False,False,True,False
2,-1.803390,-1.560203,False,False,False,False,True,True,False
3,0.392776,0.772004,False,True,False,False,False,False,True
4,-1.465519,-0.998746,False,False,False,True,False,True,False
5,0.967158,1.117516,True,False,False,False,False,False,True
6,-0.114032,0.253735,False,True,False,False,False,False,True
7,0.392776,-0.869179,False,False,True,False,False,False,True


In [57]:
categorical_columns = list(set(features.columns)-set(numeric_columns))
categorical_columns

['OverallGrade_F',
 'OverallGrade_A',
 'OverallGrade_C',
 'OverallGrade_B',
 'OverallGrade_E',
 'Obedient_Y',
 'Obedient_N']

### Criando o modelo

In [60]:
lr = LogisticRegression()
model = lr.fit(features,np.array(labels['Recommend']))

display(model.get_params())

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

### Avaliando o modelo

In [62]:
pred_labels = model.predict(features)
actual_labels = np.array(labels['Recommend'])

print('Accuracy:', float(accuracy_score(actual_labels,pred_labels))*100, '%')
print('Classification Stats:')
print(classification_report(actual_labels, pred_labels))

Accuracy: 100.0 %
Classification Stats:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [71]:
path_to_checkpoints = '/checkpoints/student_records/'
path_to_save = root_project + path_to_checkpoints
if not os.path.exists(path_to_save):
    os.mkdir(path_to_save)
jobl = joblib
jobl(model, path_to_save+'student_model.pickle')
jobl(ss, path_to_save+'student_scaler.pickle')

TypeError: 'module' object is not callable